In [1]:
import os
import yaml
import time
import optuna

from datasets import load_dataset, load_from_disk
from plaid.bridges.huggingface_bridge import huggingface_dataset_to_plaid, huggingface_description_to_problem_definition
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

from ml_pipeline_nodes_2 import PLAIDTransformedTargetRegressor, WrappedPlaidSklearnTransformer, WrappedPlaidSklearnRegressor, PLAIDFeatureTransformer

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np

import warnings
warnings.filterwarnings('ignore', module='sklearn')

nb_cpus = os.cpu_count()
print("Number of CPUs:", nb_cpus)

c:\Users\d582428\AppData\Local\miniforge3\envs\plaid-dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of CPUs: 12


In [2]:

with open("config_2.yml") as f:
    config = yaml.safe_load(f)

global_params = config["global"]


start = time.time()
hf_dataset = load_dataset(global_params['dataset_path'], split="all_samples")
# hf_dataset = load_from_disk(global_params['dataset_path'])
print(f"Loading dataset from HuggingFace Hub took: {time.time() - start:.2g} seconds")

prob_def = huggingface_description_to_problem_definition(hf_dataset.description)

train_split = prob_def.get_split(global_params['train_split_name'])[:24]
dataset_train, _ = huggingface_dataset_to_plaid(hf_dataset, ids = train_split, processes_number = int(nb_cpus/4))#os.cpu_count())

test_split = prob_def.get_split(global_params['test_split_name'])[:24]
dataset_test, _ = huggingface_dataset_to_plaid(hf_dataset, ids = test_split, processes_number = int(nb_cpus/4))#os.cpu_count())

del hf_dataset

Loading dataset from HuggingFace Hub took: 0.34 seconds
Converting huggingface dataset to plaid dataset...


100%|██████████| 24/24 [00:07<00:00,  3.32it/s]


Converting huggingface dataset to plaid dataset...


100%|██████████| 24/24 [00:07<00:00,  3.16it/s]


In [3]:
preprocessor = Pipeline([
    ('input_scalar_scaler', WrappedPlaidSklearnTransformer(MinMaxScaler(), params = config['input_scalar_scaler'])),
    ('pca_nodes', WrappedPlaidSklearnTransformer(PCA(n_components = config['pca_nodes']['n_components']), params = config['pca_nodes'])),
])
preprocessor

,steps,"[('input_scalar_scaler', ...), ('pca_nodes', ...)]"
,transform_input,None
,memory,None
,verbose,False
,sklearn_block,MinMaxScaler()
,params,"{'in_features_identifiers': [{'name': 'angle_in', 'type': 'scalar'}, {'name': 'mach_out', 'type': 'scalar'}], 'scaler_type': 'MinMaxScaler'}"
,feature_range,"(0, ...)"
,copy,True
,clip,False
,sklearn_block,PCA(n_components=3)
,params,"{'in_features_identifiers': [{'base_name': 'Base_2_2', 'type': 'nodes'}], 'n_components': 3, 'out_features_identifiers': [{'name': 'reduced_nodes_*', 'type': 'scalar'}]}"


In [4]:
preprocessor2 = PLAIDFeatureTransformer([
    ('input_scalar_scaler', WrappedPlaidSklearnTransformer(MinMaxScaler(), params = config['input_scalar_scaler']), config['input_scalar_scaler']['in_features_identifiers']),
    ('pca_nodes', WrappedPlaidSklearnTransformer(PCA(n_components = config['pca_nodes']['n_components']), params = config['pca_nodes']), config['pca_nodes']['in_features_identifiers']),
], remainder_feature_id = [config['pca_mach']['in_features_identifiers']])
preprocessor2

remainder_feature_id = [[{'type': 'field', 'name': 'mach', 'base_name': 'Base_2_2'}]]


,transformers,"[('input_scalar_scaler', ...), ('pca_nodes', ...)]"
,remainder_feature_id,"[[{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}]]"


In [5]:
postprocessor = Pipeline([
    ('pca_mach', WrappedPlaidSklearnTransformer(PCA(n_components = config['pca_mach']['n_components']), params = config['pca_mach'])),
])
postprocessor

,steps,"[('pca_mach', ...)]"
,transform_input,None
,memory,None
,verbose,False
,sklearn_block,PCA(n_components=5)
,params,"{'in_features_identifiers': [{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}], 'n_components': 5, 'out_features_identifiers': [{'name': 'reduced_mach_*', 'type': 'scalar'}]}"
,n_components,5
,copy,True
,whiten,False
,svd_solver,'auto'
,tol,0.0


In [6]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.multioutput import MultiOutputRegressor


def sklearn_regressor_constructor(X):

    available_kernel_classes = {
        "Matern":Matern
    }

    options = config['regressor_mach']['options']
    kernel_class = available_kernel_classes[options['kernel']]

    if options["anisotropic"]:
        kernel = ConstantKernel() * kernel_class(length_scale=np.ones(X.shape[1]), length_scale_bounds=(1e-8, 1e8),
                                **options["kernel_options"]) + WhiteKernel(noise_level_bounds=(1e-8, 1e8))
    else:
        kernel = kernel_class(length_scale_bounds=(1e-8, 1e8), **options["kernel_options"]) \
            + WhiteKernel(noise_level_bounds=(1e-8, 1e8))

    gpr = GaussianProcessRegressor(
        kernel=kernel,
        optimizer=options["optim"],
        n_restarts_optimizer=options["num_restarts"],
        random_state=options["random_state"])

    return MultiOutputRegressor(gpr)

regressor = PLAIDTransformedTargetRegressor(
    regressor=WrappedPlaidSklearnRegressor(sklearn_regressor_constructor, params = config['regressor_mach']),
    transformer=postprocessor,
)
regressor

,regressor,WrappedPlaidS...egressor(None)
,transformer,Pipeline(step...ponents=5)))])
,sklearn_block_constructor,<function skl...001E75D2AAD40>
,params,"{'in_features_identifiers': [{'name': 'angle_in', 'type': 'scalar'}, {'name': 'mach_out', 'type': 'scalar'}, ...], 'options': {'anisotropic': True, 'kernel': 'Matern', 'kernel_options': {'nu': 2.5}, 'num_restarts': 2, ...}, 'out_features_identifiers': [{'name': 'reduced_mach_*', 'type': 'scalar'}], 'type': 'GaussianProcessRegressor'}"
,sklearn_block,PCA(n_components=5)
,params,"{'in_features_identifiers': [{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}], 'n_components': 5, 'out_features_identifiers': [{'name': 'reduced_mach_*', 'type': 'scalar'}]}"
,n_components,5
,copy,True
,whiten,False
,svd_solver,'auto'
,tol,0.0


In [7]:
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor2),
        ("regressor", regressor),
    ]
)
pipeline

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('input_scalar_scaler', ...), ('pca_nodes', ...)]"
,remainder_feature_id,"[[{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}]]"
,regressor,WrappedPlaidS...egressor(None)
,transformer,Pipeline(step...ponents=5)))])
,sklearn_block_constructor,<function skl...001E75D2AAD40>
,params,"{'in_features_identifiers': [{'name': 'angle_in', 'type': 'scalar'}, {'name': 'mach_out', 'type': 'scalar'}, ...], 'options': {'anisotropic': True, 'kernel': 'Matern', 'kernel_options': {'nu': 2.5}, 'num_restarts': 2, ...}, 'out_features_identifiers': [{'name': 'reduced_mach_*', 'type': 'scalar'}], 'type': 'GaussianProcessRegressor'}"
,sklearn_block,PCA(n_components=5)


In [8]:
pipeline.fit(dataset_train)
dataset_pred = pipeline.predict(dataset_train)
dataset_pred[0].get_field("mach", base_name = "Base_2_2")

TypeError: list indices must be integers or slices, not str

In [ ]:
# print(pipeline.get_params())
print(pipeline.get_params()['preprocessor__pca_nodes__sklearn_block__n_components'])
print(pipeline.get_params()['regressor__transformer__pca_mach__sklearn_block__n_components'])

3
5


In [ ]:
def objective(trial):
    # Suggest hyperparameters
    nodes_n_components = trial.suggest_int("preprocessor__pca_nodes__sklearn_block__n_components", 2, 5)
    mach_n_components = trial.suggest_int("regressor__transformer__pca_mach__sklearn_block__n_components", 4, 12)

    # Clone and configure pipeline
    pipeline_run = clone(pipeline)
    pipeline_run.set_params(
        preprocessor__pca_nodes__sklearn_block__n_components=nodes_n_components,
        regressor__transformer__pca_mach__sklearn_block__n_components=mach_n_components
    )

    cv = KFold(n_splits=3, shuffle=True, random_state=42)

    scores = []

    indices = np.arange(len(dataset_train))

    for train_idx, val_idx in cv.split(indices):

        dataset_cv_train_ = dataset_train[train_idx]
        dataset_cv_val_   = dataset_train[val_idx]

        pipeline_run.fit(dataset_cv_train_)

        predicted_dataset_cv_val_ = pipeline_run.predict(dataset_cv_val_)

        score = pipeline_run.score(dataset_cv_val_, predicted_dataset_cv_val_)

        scores.append(score)

    return np.mean(scores)


In [ ]:
print("dataset[0].get_field() =", dataset_train[0].get_field("mach", base_name="Base_2_2"))

dataset[0].get_field() = [0.39387196 0.39389698 0.39392865 ... 0.81002502 0.81000822 0.80999194]


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=2)

[I 2025-07-21 16:04:25,779] A new study created in memory with name: no-name-74153c1b-c11d-4a84-9e8b-106ebff6edb7
[I 2025-07-21 16:04:28,731] Trial 0 finished with value: 0.07025298219725497 and parameters: {'preprocessor__pca_nodes__sklearn_block__n_components': 4, 'regressor__transformer__pca_mach__sklearn_block__n_components': 10}. Best is trial 0 with value: 0.07025298219725497.
[I 2025-07-21 16:04:30,750] Trial 1 finished with value: 0.07117518543954957 and parameters: {'preprocessor__pca_nodes__sklearn_block__n_components': 5, 'regressor__transformer__pca_mach__sklearn_block__n_components': 4}. Best is trial 0 with value: 0.07025298219725497.


In [ ]:
params = {'preprocessor__pca_nodes__sklearn_block__n_components': 4, 'regressor__transformer__pca_mach__sklearn_block__n_components': 6}
index = 5
optimized_pipeline = clone(pipeline)
# optimized_pipeline.set_params(**study.best_params)
optimized_pipeline.set_params(**params)
optimized_pipeline.fit(dataset_train)
dataset_pred = optimized_pipeline.predict(dataset_train)
print("mach_pred =", dataset_pred[index].get_field("mach", base_name = "Base_2_2"))
print("mach_ref =", dataset_train[index].get_field("mach", base_name = "Base_2_2"))
print("--")
print("angle_in =", dataset_pred[index].get_scalar("angle_in"))
print("mach_out =", dataset_pred[index].get_scalar("mach_out"))
print("reduced_nodes_* =", [dataset_pred[index].get_scalar("reduced_nodes_*")[j] for j in range(params['preprocessor__pca_nodes__sklearn_block__n_components'])])

mach_pred = [0.3389678  0.33901469 0.3390761  ... 0.7710585  0.77117304 0.77128376]
mach_ref = [0.3411916  0.34125071 0.34132835 ... 0.78406136 0.78400726 0.78394869]
--
angle_in = 0.6386494252873546
mach_out = 0.15684931506849198
reduced_nodes_* = [-31.132563952427006, 26.350272966900363, -0.9765460147618381, -0.9322534183224835]


In [ ]:
print(optimized_pipeline.named_steps["regressor"].regressor.sklearn_block.estimators_[0].kernel_.get_params()['k1__k2__length_scale'])#.kernel_.length_scale

[4.57950198e+00 2.42415860e+00 2.50716442e+02 5.97813291e+01
 1.86570867e+05 3.93160802e+01]


In [ ]:
dataset_pred_2 = optimized_pipeline.predict(dataset_train)
optimized_pipeline.score(dataset_train, dataset_pred_2)

0.026628778055004736

In [ ]:
params = {'pca_nodes__sklearn_block__n_components': 4}
index = 5
preprocessor_ = clone(preprocessor)
preprocessor_.set_params(**params)
dataset_pred = preprocessor_.fit_transform(dataset_train)
print("angle_in =", dataset_pred[index].get_scalar("angle_in"))
print("mach_out =", dataset_pred[index].get_scalar("mach_out"))
print("reduced_nodes_* =", dataset_pred[index].get_scalar("reduced_nodes_*"))

angle_in = 0.6386494252873546
mach_out = 0.15684931506849198
reduced_nodes_* = [-31.13256395  26.35027297  -0.97654601  -0.93225342]


# COMMENTED TESTS

In [ ]:
# test = WrappedPlaidSklearnTransformer(MinMaxScaler(), params = config['input_scalar_scaler'])
# test.fit(dataset_train)

In [ ]:
# dd = test.transform(dataset_train)

# print(dataset_train)
# print(dd)

# dataset_2 = test.inverse_transform(dd)

# print(dataset_train[0].get_scalar("angle_in"))
# print(dd[0].get_scalar("angle_in"))
# print(dataset_2[0].get_scalar("angle_in"))
# print('===')
# print(dataset_train[0].get_scalar("mach_out"))
# print(dd[0].get_scalar("mach_out"))
# print(dataset_2[0].get_scalar("mach_out"))

In [ ]:
# pca = WrappedPlaidSklearnTransformer(PCA(n_components = config['pca_nodes']['n_components']), params = config['pca_nodes'])
# pca.fit(dataset_train)
# dd = pca.transform(dataset_train)
# dataset_2 = pca.inverse_transform(dd)

# print(dataset_train[0].get_nodes(base_name = "Base_2_2"))
# print(dd[0].get_scalar("reduced_nodes_*"))
# print(dataset_2[0].get_nodes(base_name = "Base_2_2"))

In [ ]:
# pca2 = WrappedPlaidSklearnTransformer(PCA(n_components = config['pca_mach']['n_components']), params = config['pca_mach'])
# pca2.fit(dataset_train)
# dd = pca2.transform(dataset_train)
# dataset_2 = pca2.inverse_transform(dd)

# print(dataset_train[0].get_field("mach", base_name = "Base_2_2"))
# print(dd[0].get_scalar("reduced_mach_*"))
# print(dataset_2[0].get_field("mach", base_name = "Base_2_2"))